In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#DATABASE_URL = "sqlite:///:memory:"
#DATABASE_URL = "sqlite:///example.db"
#DATABASE_URL = "mysql://root@localhost/braindocs"

In [3]:
from __future__ import print_function
import dateutil.parser
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from bdAPI import bdAPI
from database import Base, AnalysisResult, AnalysisResultTextUnit, session_scope
from configparser import PromptingConfigParser, promptyesno

In [8]:
print("braindocs2mysql")
print("Copyright (c) 2015 ai-one inc. All rights reserved.")
print("")
print(">>> Current settings:\n")

cfg = PromptingConfigParser()

# add default settings
cfg.add_section("braindocs")
cfg.add_section("database")
cfg.set("database", "url", "mysql://root@localhost/braindocs")
cfg.set("braindocs", "username", "bd_user")
cfg.set("braindocs", "password", "bd_pw")
cfg.set("braindocs", "url", "https://nathandev2.cloudapp.net/at")

# read settings from file
cfg.read("braindocs2database.config")

# prompt update
cfg.print()
while promptyesno("\nUpdate settings?"):
    print("")
    cfg.prompt()
    with open("braindocs2mysql.config", 'wb') as fp:
        cfg.write(fp)
    print("\n>>> Updated settings:\n")
    cfg.print()


braindocs2mysql
Copyright (c) 2015 ai-one inc. All rights reserved.

>>> Current settings:

BRAINDOCS
   username: tom
   password: braindocs
   url: https://nathandev2.cloudapp.net/at
DATABASE
   url: sqlite:///:memory:

Update settings? [no]? y

BRAINDOCS
   Specify username [tom]: tmarsh
   Specify password [braindocs]: 
   Specify url [https://nathandev2.cloudapp.net/at]: 
DATABASE
   Specify url [sqlite:///:memory:]: 

>>> Updated settings:

BRAINDOCS
   username: tmarsh
   password: braindocs
   url: https://nathandev2.cloudapp.net/at
DATABASE
   url: sqlite:///:memory:

Update settings? [no]? 


In [9]:
engine = create_engine(cfg.get("database", "url"), echo=False)

Base.metadata.drop_all(engine) # optional: drop all
Base.metadata.create_all(engine)
 
Session = sessionmaker(bind=engine)

In [10]:
bd = bdAPI(
    username=cfg.get("braindocs", "username"),
    password=cfg.get("braindocs", "password"),
    baseURL=cfg.get("braindocs", "url"))

with session_scope(Session()) as session:
    for ar in bd.getAnalysisResults():
        arentry = AnalysisResult()
        for key, value in ar.iteritems():
            if key in ["customKeywords", "userid"]:
                if not value is None: 
                    value = ",".join(v.replace(",", "\\,") for v in value)
            if key == "create_date":
                value = dateutil.parser.parse(value).replace(tzinfo=None)
            setattr(arentry, key, value)
            if key == "analysisId":
                for tu in bd.getAnalysisDetailsTextUnits(value):
                    tuentry = AnalysisResultTextUnit()
                    for key, value in tu.iteritems():
                        setattr(tuentry, key, value)
                    arentry.textunits.append(tuentry)
        session.merge(arentry)

In [11]:
with session_scope(Session()) as session:
    print(session.query(AnalysisResult).filter_by(agentName=u"Wireless Sensor Agent 19930073118 v3").first())

<AnalysisResult(_id='55bd3ea391c8a9c94f6dfef8', name=Wireless Sensor 5th Test, textunits=[[<AnalysisResultTextUnit(_id='55bd3ea8473c203221fcb7a8', score=1.48342871666, fingerprint_id=1271, ..., text=Micro-Elec...)>, <AnalysisResultTextUnit(_id='55bd3ea8473c203221fcb45a', score=1.46657776833, fingerprint_id=425, ..., text=The Nation...)>, <AnalysisResultTextUnit(_id='55bd3ea8473c203221fcb76d', score=1.46597653627, fingerprint_id=1212, ..., text=This volum...)>, <AnalysisResultTextUnit(_id='55bd3ea8473c203221fcb439', score=1.46164017916, fingerprint_id=392, ..., text=The Small ...)>, <AnalysisResultTextUnit(_id='55bd3ea8473c203221fcb7a9', score=1.45471739769, fingerprint_id=1272, ..., text=Biological...)>, <AnalysisResultTextUnit(_id='55bd3ea8473c203221fcb36e', score=1.44295448065, fingerprint_id=189, ..., text=The chemic...)>, <AnalysisResultTextUnit(_id='55bd3ea8473c203221fcb7cd', score=1.40430647135, fingerprint_id=1308, ..., text=This paper...)>, <AnalysisResultTextUnit(_id='55bd3ea8